# Programa auxiliar para separar imágenes de Entrenamiento y Prueba distribuyendolas al azar

In [ ]:
#@title Librerías a usar
import os
import os.path
import numpy as np
import shutil

print("\nLibrerías importadas")


Librerías importadas


In [ ]:
#@title Acceder al Drive
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS' #@param {type:"string"}
path_fuente = '/all'  #@param {type:"string"}
path_destino_entrenamiento = '/train'  #@param {type:"string"}
path_destino_prueba = '/test'  #@param {type:"string"}

imagPath = path + path_fuente
imagPath_train = path + path_destino_entrenamiento
imagPath_test = path + path_destino_prueba

Mounted at /content/gdrive


In [ ]:
#@title Levantar carpeta de imágenes a distribuir

# procesa los archicos
todasImags = {}

print("Procesando ",imagPath,"...\n")

all_dirs = os.listdir( imagPath )
for clase in all_dirs:

    auxiPath = imagPath + '/' + clase

    # levanta la lista de imágenes de cada letra
    todasImags[clase] = [ fn for fn in os.listdir( auxiPath ) if fn.endswith('.png') or fn.endswith('.jpg') ]
    # desordena al azar
    np.random.shuffle( todasImags[clase] )

    print("Levanta de ", clase, ": ", len(todasImags[clase])," imágenes.")



Procesando  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/all ...

Levanta de  0 :  30  imágenes.
Levanta de  4 :  30  imágenes.
Levanta de  5 :  30  imágenes.
Levanta de  6 :  48  imágenes.
Levanta de  8 :  30  imágenes.
Levanta de  1 :  30  imágenes.
Levanta de  9 :  30  imágenes.
Levanta de  7 :  30  imágenes.
Levanta de  2 :  30  imágenes.
Levanta de  3 :  30  imágenes.


In [ ]:
#@title Normaliza cantidades por clase (opcional)
unificar_cantid_por_clase = False #@param {type:"boolean"}

if unificar_cantid_por_clase:
  # determina cantidad mínima
  min = None
  for clase in todasImags.keys():
     cantImg = len(todasImags[clase])
     if (min is None) or (min > cantImg):
       min = cantImg

  print("Cantidad a unificar: ", min)
  print("\n")
  # unifica esa cantidad como mínima
  for clase in todasImags.keys():
    todasImags[clase] = np.random.choice(todasImags[clase], size=min, replace=False)

    print("Nueva cantidad de ", clase, ": ", len(todasImags[clase])," imágenes.")


In [ ]:
#@title Determina Cantidades a distribuir

porc_Entrenamiento = 80 #@param {type:"slider", min:10, max:100, step:1}
if porc_Entrenamiento < 10:
  porc_Entrenamiento = 10

# muestra proporciones a usar
for clase in todasImags.keys():
  cantImg = len(todasImags[clase])
  cantE = ( cantImg * porc_Entrenamiento ) // 100
  cantP = cantImg - cantE

  print('\nImágenes de ' , clase, ":")
  print('- Imágenes para Entrenamiento: ', cantE)
  print('- Imágenes para Validación: ', cantP)


Imágenes de  0 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  4 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  5 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  6 :
- Imágenes para Entrenamiento:  38
- Imágenes para Validación:  10

Imágenes de  8 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  1 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  9 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  7 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  2 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6

Imágenes de  3 :
- Imágenes para Entrenamiento:  24
- Imágenes para Validación:  6


In [ ]:
#@title Mueve archivos para Entrenamiento

def moverArchivos(imagPathOrigen_Clase, arFNClase, imagPathDestino_Clase):
  cant = 0
  for name in arFNClase:
      # mueve la imagen
      if os.path.isfile(imagPathDestino_Clase + '/' + name):
        print("**", name, " ya existe en ", imagPathDestino_Clase)
      else:
        shutil.move(imagPathOrigen_Clase + '/' + name, imagPathDestino_Clase)
        cant = cant + 1
  print("\t", cant, " imágenes movidas a ", imagPathDestino_Clase)
  return cant


# crea directorios, si es necesario
if not os.path.isdir(imagPath_train):
    os.makedirs(imagPath_train)
if not os.path.isdir(imagPath_test):
    os.makedirs(imagPath_test)

for clase in todasImags.keys():

  print("> ", clase, ":")
  cantE = 0
  cantP = 0
  # crea subdirectorios, si es necesario
  imagpathOrigenClase = imagPath + '/' + clase
  imagPath_trainClase = imagPath_train + '/' + clase
  imagPath_testClase = imagPath_test + '/' + clase
  if not os.path.isdir(imagPath_trainClase):
      os.makedirs(imagPath_trainClase)
  if not os.path.isdir(imagPath_testClase):
      os.makedirs(imagPath_testClase)

  # determina cantidades
  arImgsFN = np.array(todasImags[clase])
  cantImg = len(todasImags[clase])
  cantE = ( cantImg * porc_Entrenamiento ) // 100
  #cantP = cantImg - cantE

  # separa
  train_FileNames, test_FileNames = np.split(arImgsFN, [cantE])

  # mueve imágenes
  moverArchivos(imagpathOrigenClase, train_FileNames, imagPath_trainClase)
  moverArchivos(imagpathOrigenClase, test_FileNames, imagPath_testClase)


>  0 :
	 24  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/0
	 6  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/test/0
>  4 :
	 24  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/4
	 6  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/test/4
>  5 :
	 24  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/5
	 6  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/test/5
>  6 :
	 38  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/6
	 10  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/test/6
>  8 :
	 24  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/8
	 6  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/test/8
>  1 :
	 24  imágenes movidas a  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/1
	 6  imágenes movidas

In [ ]:
#@title Fuerza actualización del drive (sino no aparecen los nuevos archivos)
drive.flush_and_unmount()
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#@title Muestra nuevas cantidades
def contarImagenes(imagPath):

  print("\n\n> procesando ", imagPath, ": ")

  # procesa los subdirectorios con las imágenes
  all_dirs = os.listdir( imagPath )
  for each_dir in all_dirs:

    auxiPath = imagPath + '/' + each_dir

    # obtiene la cantidad de imágenes disponibles
    cantImagDir = len(os.listdir( auxiPath ))

    print("  -- ", auxiPath, ": tiene ", cantImagDir, " imágenes.")

# realiza conteo
contarImagenes(imagPath_train)
contarImagenes(imagPath_test)



> procesando  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train : 
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/0 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/4 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/5 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/6 : tiene  38  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/8 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/1 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/9 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/7 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/2 : tiene  24  imágenes.
  --  /content/gdrive/My Drive/IA/demoML/imagenes/NUMEROS/train/3 : tiene  24  imágenes.


> procesando  /content/gdrive/M